# Stress Constrained TO Algorithm

Conceptualised and developed by erin.yu22@imperial.ac.uk, sasha.halsey20@imperial.ac.uk and a.panesar@imperial.ac.uk

-------------------
### Objectives of this lab

1) Add a stress constraint to the TO problem
2) Compare to previous TO results

####  Import library

Run the cell below to import necessarry python libraries for this lab.

If you encounter an error, identify the unsuccessfully-installed library name by reading the error message, cut the corresponding pip install statement (without '#' symbol) into a separate cell and run it. Upon running the pip install cell, then restart the kernel and run the (default) import library cell again. Raise hand during the lab or email erin.yu22@imperial.ac.uk if you cannot resolve the issue.

In [ ]:
from __future__ import division
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from matplotlib import colors
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import spsolve

-----------
# The P-Norm Stress Aggregation
We all want a little less stress in our lives, but how much? A loaded structure will have some stress, no matter what. If we changed our problem to 'minimise stress' as the objective, we would end up with no structure - the least stressed structure is no structure at all!

Instead, we shall add stress as a constraint in a similar way to volume / weight, using a limiting factor. You might be thinking - "let's limit it to yield stress". But checking the stress of every element is computationally expensive and is not differentiable, so we can't get the sensitivity value out of it. Therefore we use an approximation for large stresses in the design field. One way is called p-norm stress aggregation, which sums up all the local stress values into a single global value. Similar to SIMP, it uses a penalty-like factor called p-norm, we raise the elemental stress to it's power, sum these values up across the domain and then inverse the power to get a more sensible value. We can increase the value of p-norm to increase the contribution of higher stresses, and as p-norm tends to infinity, the value of the aggregation tends towards the maximum value.

$$
p_{\text{stress}} = \left( \frac{1}{N_\text{el}} \sum_{e=1}^{N_\text{el}} \sigma_{\text{vm},e}^{\,p_\text{norm}} \right)^{\frac{1}{p_\text{norm}}}
$$

where:  

- $N_\text{el}$ = total number of finite elements  
- $\sigma_{\text{vm},e}$ = von Mises stress of element $e$  
- $p_\text{norm}$ = chosen p-norm exponent (typically 8–20), but we will play around with this later.


Have a play around with some arrays of varying distributions (similar to density) to see the effect the values and p-norm values have on the p-norm aggregation:


In [ ]:
# 1) Mostly around 0.2  
x1 = np.array([0.263, 0.196, 0.188, 0.215, 0.224, 0.140, 0.200, 0.201, 0.182, 0.178])
print("x1 :", x1)

# 2) Mostly around 0.5
x2 = np.array([0.567, 0.532, 0.546, 0.468, 0.596, 0.520, 0.460, 0.522, 0.461, 0.436])
print("x2 :", x2)

# 3) Mostly small, a few entries = 1.0
x3 = np.array([0.059, 0.044, 0.085, 1.000, 0.045, 0.010, 1.000, 1.000, 0.055, 0.019])
print("x3 :", x3)

# 4) Uniform spread from 0 → 1
x4 = np.array([0.000, 0.111, 0.222, 0.333, 0.444, 0.556, 0.667, 0.778, 0.889, 1.000])
print("x4 :", x4)

# 5) Mostly around 0.9
x5 = np.array([0.863, 0.959, 0.902, 0.909, 0.840, 0.947, 0.895, 0.978, 0.888, 0.932])
print("x5 :", x5)

def pnorm(x, p):
    return ((1/10)*np.sum(x**p))**(1.0/p)

p_values = [1, 2, 4, 8, 16]
results = {}

for p in p_values:
    results[p] = {"x1": pnorm(x1, p), "x2": pnorm(x2, p), "x3": pnorm(x3, p), "x4": pnorm(x4, p), "x5": pnorm(x5, p),}
for p in p_values:
    print(f"\np = {p}")
    print(f"  x1 (mostly 0.2):           {results[p]['x1']:.3f}")
    print(f"  x2 (mostly 0.5):           {results[p]['x2']:.3f}")
    print(f"  x3 (few at 1.0):           {results[p]['x3']:.3f}")
    print(f"  x4 (uniform   ):           {results[p]['x4']:.3f}")
    print(f"  x5 (mostly 0.9):           {results[p]['x5']:.3f}")

-----------
# 1) Adding Another Constraint to the Problem
### Optimisation Framework

Previously we solved a density–based topology optimisation problem with the SIMP formulation.

- **Design variable:**  
  Elemental relative density  
  $$ x_e \in [0,1] $$

- **Objective function:**  
  Minimise compliance (maximise stiffness)
  $$
  \min_x\; C(x) = \mathbf{F}^\mathsf{T}\mathbf{U}(x)
  $$
  where the displacement field satisfies  
  $$
  \mathbf{K}(x)\,\mathbf{U}(x) = \mathbf{F}.
  $$

- **Constraint:**  
  Global volume fraction  
  $$
  \frac{1}{N_e}\sum_{e=1}^{N_e} x_e = V^\ast.
  $$

  In this lab we will formulate the p-norm stress constraint as can then be written as:

  $$
  P_{\rm stress} \le \sigma_{\rm limit}
  $$

  where $\sigma_\text{allow}$ is the allowed stress (e.g. 90% of maximum).  
  This ensures that the structural stress does not exceed the specified limit. In this formulation we will add this as $c_2$ which is the difference between $P_{\rm stress}$ and $\sigma_{\rm limit}$, we want this difference to tend towards 0 to reduce the p-norm stress integral.

  $$
  c_2 = P_{\rm stress} - \sigma_{\rm limit}
  $$



# 1.1) Input parameters

In [ ]:
from setup_topology import setup_topology_stress
from iterate_topology import iterate_topology_stress

params = dict(nelx=30, nely=20, rmin=1, Emin=1e-9, Emax=1.0, nu=0.3, penal=3, volfrac=0.3, sigma_max=0.3, p_norm=8)
state = setup_topology_stress(**params)

# 1.2) New optimisation loop with additional constraint defined
The only thing you have to worry about here is the c2_limit and max_stress_weight which you can change to see it's effect on convergence and resulting design

In [ ]:
def iterate_topology(state, params, c2_limit=0.1, max_stress_weight=1000.0):
    # Unpack state
    nelx, nely = state['nelx'], state['nely']
    KE = state['KE']; edofMat = state['edofMat']
    iK = state['iK']; jK = state['jK']
    free = state['free']; f = state['f']; u = state['u']
    H, Hs = state['H'], state['Hs']
    x = state['x']; xPhys = state['xPhys']
    Emin = params.get('Emin', state['Emin'])
    Emax = params.get('Emax', state['Emax'])
    penal = params.get('penal', state['penal'])
    ndof = state['ndof']

    # --- 1) Solve FE system ---
    sK = ((KE.flatten()[np.newaxis]).T * (Emin + xPhys**penal * (Emax - Emin))).flatten(order='F')
    K = coo_matrix((sK, (iK, jK)), shape=(ndof, ndof)).tocsc()
    K_reduced = K[free, :][:, free]
    u[free, 0] = spsolve(K_reduced, f[free, 0])

    # --- 2) Objective: strain energy ---
    ue = u[edofMat].reshape(nelx*nely, 8)
    ce = (np.dot(ue, KE) * ue).sum(axis=1)
    obj = ((Emin + xPhys**penal * (Emax - Emin)) * ce).sum()

    # --- 3) Objective sensitivity ---
    dc = (-penal * xPhys**(penal-1) * (Emax - Emin)) * ce
    dv = np.ones_like(dc)

    # --- 4) Filter objective sensitivities ---
    dc = np.asarray(H @ (dc[np.newaxis].T / Hs))[:, 0]
    dv = np.asarray(H @ (dv[np.newaxis].T / Hs))[:, 0]

    # --- 5) Compute stress & p-norm sensitivity ---
    # — Compute stress & p-norm sensitivity —
    def stress_and_pnorm(state, eps_vm=1e-12, eps_S=1e-12):
        xPhys = state['xPhys']
        u = state['u'][:,0]
        penal = state['penal']
        KE = state['KE']
        edofMat = state['edofMat']
        H = state['H']; Hs = state['Hs']
        nelx, nely = state['nelx'], state['nely']
        nel = nelx*nely
        Emin, Emax = state['Emin'], state['Emax']
        nu = state['nu']
        sigma_max = state['sigma_max']
        p_norm = state['p_norm']
    
        D_unit = (1.0 / (1.0 - nu**2)) * np.array([[1, nu, 0],
                                                    [nu, 1, 0],
                                                    [0, 0, (1-nu)/2]])
        B_center = 0.25*np.array([[-1,0,1,0,1,0,-1,0],
                                  [0,-1,0,-1,0,1,0,1],
                                  [-1,-1,-1,1,1,1,1,-1]])
    
        sigma_elem = np.zeros((nel, 3))
        sigma_vm = np.zeros(nel)
        eps_elem = np.zeros((nel, 3))
    
        for e in range(nel):
            u_e = u[edofMat[e]]
            eps_e = B_center @ u_e
            eps_elem[e,:] = eps_e
            E_e = Emin + xPhys[e]**penal*(Emax-Emin)
            sig_e = E_e * D_unit @ eps_e
            sigma_elem[e,:] = sig_e
            sx, sy, txy = sig_e
            sigma_vm[e] = np.sqrt(sx**2 - sx*sy + sy**2 + 3*txy**2)
    
        vm_safe = np.maximum(sigma_vm, eps_vm)
        S = np.sum(vm_safe**p_norm)/nel + eps_S
        p_stress = S**(1.0/p_norm)
        c2 = p_stress - sigma_max
    
        # sensitivity
        dE_dx = penal * xPhys**(penal-1)*(Emax-Emin)
        prefactor = (1.0/nel) * S**(1.0/p_norm - 1.0)
        dc2 = np.zeros(nel)
        for e in range(nel):
            dsig_dE = D_unit @ eps_elem[e]
            dsig_dx = dsig_dE * dE_dx[e]
            sx, sy, txy = sigma_elem[e]
            vm = vm_safe[e]
            dvm_dsig = np.array([(2*sx - sy)/(2*vm), (2*sy - sx)/(2*vm), 3*txy/vm])
            dvm_dx = dvm_dsig @ dsig_dx
            dc2[e] = prefactor * (vm**(p_norm-1)) * dvm_dx
    
        # filter
        dc2 = np.asarray(H * (dc2[np.newaxis].T / Hs))[:,0]
        return sigma_vm, p_stress, c2, dc2
    sigma_vm, p_stress, c2, dc2 = stress_and_pnorm(state)
    state['sigma_vm'] = sigma_vm
    state['p_stress'] = p_stress
    state['c2'] = c2
    state['dc2'] = dc2

    # — OC update —
    def oc_update(nelx, nely, x, volfrac, dc, dv, g):
        l1, l2 = 0.0, 1e9
        move = 0.2
        xnew = np.zeros_like(x)
        eps = 1e-9
        for _ in range(80):
            lmid = 0.5*(l1 + l2)
            with np.errstate(divide='ignore', invalid='ignore'):
                arg = np.maximum(1e-9, -dc/dv / lmid)
                x_candidate = np.maximum(0.0, np.maximum(x - move,
                                     np.minimum(1.0, np.minimum(x + move, x*np.sqrt(arg)))))
            if np.sum(x_candidate) - volfrac*nelx*nely > 0:
                l1 = lmid
            else:
                l2 = lmid
            xnew[:] = x_candidate
            if abs(l2-l1)/(l1+l2+eps) < 1e-3:
                break
        return xnew, np.sum(x_candidate) - volfrac*nelx*nely
        
    # --- 6) Automatically enforce stress constraint ---
    stress_weight = 1.0
    max_iter = 10
    for _ in range(max_iter):
        # Combine sensitivities
        dc_total = dc + stress_weight * dc2

        # OC update
        xnew, _ = oc_update(nelx, nely, x, state['volfrac'], dc_total, dv, c2)

        # Recompute stress for candidate design
        state['x'] = xnew
        _, p_stress_new, c2_new, _ = stress_and_pnorm(state)

        # Check if stress is below limit
        if c2_new <= c2_limit or stress_weight >= max_stress_weight:
            break

        # Increase stress weight to penalise violating elements
        stress_weight *= 2.0

    # --- 7) Update state ---
    state['xold'] = x.copy()
    state['x'] = xnew
    xPhys_filtered = np.asarray(H @ (xnew[np.newaxis].T / Hs))[:, 0]
    state['xPhys'] = xPhys_filtered
    state['ce'] = ce
    state['dc'] = dc
    state['dv'] = dv

    change = np.max(np.abs(state['x'] - state['xold']))
    info = {
        'obj': float(obj),
        'change': float(change),
        'p_stress': float(p_stress_new),
        'c2': float(c2_new)
    }
    return state, info

# 1.3) Iterate



In [ ]:
max_iter = 50
for it in range(max_iter):
    state, info = iterate_topology(state, params)
    print(f"it {it:3d} obj {info['obj']:.4f} change {info['change']:.4f} "
          f"p_stress {info['p_stress']:.4f} c2 {info['c2']:.4f}")
    if info['change'] < 1e-3:
        break

# 1.4) Plot

In [ ]:
# Prepare plots
density_grid = -state['xPhys'].reshape((state['nelx'], state['nely'])).T
sigma_vm_grid = np.flip(state['sigma_vm'].reshape((state['nelx'], state['nely'])).T, axis=0)

# Create side-by-side axes
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))

# Density field
im1 = ax1.imshow(density_grid, cmap='gray', interpolation='none',
                 norm=colors.Normalize(vmin=-1, vmax=0))
cbar1 = fig.colorbar(im1, ax=ax1, fraction=0.03, pad=0.05)
cbar1.set_label('Relative density')
ax1.set_xticks([])
ax1.set_yticks([])
ax1.set_title("Density field")

# Stress field
im2 = ax2.imshow(sigma_vm_grid, cmap="inferno", interpolation="none", origin="lower")
cbar2 = fig.colorbar(im2, ax=ax2, fraction=0.03, pad=0.05)
cbar2.set_label("Von Mises stress")
ax2.set_xticks([])
ax2.set_yticks([])
ax2.set_title("Stress distribution")

plt.tight_layout()
plt.show()

----------
# 2) Comparing - Your Turn!

Now try running again, but changing parameters such as c2_limit and max_stress_weight to see how the convergence and design changes. 
Keep an eye on the values of p_norm aggregation, c2, obj, as you may need more iterations or relaxing of constraints to ensure convergence.

![ParetoFront](attachment:ParetoFront.png)